In [118]:
import sys
import inspect
import pandas as pd

from os import listdir
from datetime import datetime
from calendar import monthrange
from os.path import isfile, join

from xlrd import XLRDError


In [2]:
sys.path.append("..") 

from api.utilies import fileworker as fw

In [3]:
worker = fw.Worker()

In [4]:
path = '../data/raw'

In [125]:
russian_months = [  'январь', 'февраль', 'март',
                    'апрель', 'май', 'июнь', 'июль',
                    'август', 'сентябрь', 'октябрь',
                    'ноябрь', 'декабрь' ]

def filter(f, key_word='ССПС'):
    if f.find(key_word) > -1:
        if f.find('~$') == -1:
            return isfile(join(path, f))


def parse_dt(dt):
    if isinstance(dt, str):
        return datetime.strptime(dt, '%d.%m.%Y')
    else:
        return dt

def find_dt(string):
    return string.split()[3]

In [50]:
def read_apvo(file_name, path='../data/raw'):
    file_path = path + '/' + file_name
    names = ['company', 'type', 'volume', 'work_area', 'work_kind', 'work_done_phis', 'work_done_pas', 'window']

    desc = pd.read_excel(file_path, nrows=2, sheet_name=0)
    #print(type(desc.iloc[1,0]), desc.iloc[1,0])
    report_dt = parse_dt(desc.iloc[1,0])

    year = report_dt.year
    month = report_dt.month

    max_day = monthrange(year, month)[1]

    if russian_months[month-1] != file_name.split()[1]:
        print(russian_months[month-1] , file_name.split()[1])
        return 'ERROR'

    data = pd.read_excel(file_path, skiprows=[1,2,3,4], sheet_name=0, names=names)
    data.drop(data.tail(3).index, inplace=True)
    data['date'] = report_dt


    print('File name: {0}, first day: {1}, max days in month: {2}'.format(file_path, report_dt, max_day))

    for day in range(1, max_day):
            desc = pd.read_excel(file_path, nrows=2, sheet_name=day)
            report_dt = parse_dt(desc.iloc[1,0])
            #print(day, report_dt)
            day_data = pd.read_excel(file_path, skiprows=[1,2,3,4], sheet_name=day, names=names)
            day_data.drop(day_data.tail(3).index, inplace=True)
            day_data['date'] = report_dt

            data = pd.concat([data, day_data])

    data = data.reset_index(drop=True)

    return data

In [160]:
def read_ssps(file_name, path='../data/raw'):
    file_path = path + '/' + file_name
    names = ['company', 'machine', 'au12', 'rate_norm', 'rate_fact', 'low_rate', 'up_rate', 'residue']
    sheet_number = 1
    use_int = False
    sheet_name = str(sheet_number).zfill(2)
    #print(file_name)

    try:
        desc = pd.read_excel(file_path, nrows=1, sheet_name=sheet_name)
    except XLRDError:
        sheet_number = 0
        use_int = True
        desc = pd.read_excel(file_path, nrows=1, sheet_name=sheet_number)

    
    dt = find_dt(desc.iloc[0,0])
    report_dt = parse_dt(dt)
    #print(file_name, dt, report_dt)

    year = report_dt.year
    month = report_dt.month

    max_day = monthrange(year, month)[1]

    if russian_months[month-1] != file_name.split()[2]:
        print(russian_months[month-1] , file_name.split()[2])
        return 'ERROR'

    print('File name: {0}, first day: {1}, max days in month: {2}'.format(file_path, report_dt, max_day))

    if use_int:
        data = pd.read_excel(file_path, skiprows=[1,2,3,4,5,6,7], sheet_name=sheet_number, names=names)
    else:
        data = pd.read_excel(file_path, skiprows=[1,2,3,4,5,6,7], sheet_name=sheet_name, names=names)

    data['date'] = report_dt

    for day in range(sheet_number+1, max_day+sheet_number):
        try:
            if use_int:
                desc = pd.read_excel(file_path, nrows=1, sheet_name=day)
            else:
                sheet_name = str(day).zfill(2)
                desc = pd.read_excel(file_path, nrows=1, sheet_name=sheet_name)

            dt = find_dt(desc.iloc[0,0])
            report_dt = parse_dt(dt)
            
            if use_int:
                day_data = pd.read_excel(file_path, skiprows=[1,2,3,4,5,6,7], sheet_name=day, names=names)
            else:
                day_data = pd.read_excel(file_path, skiprows=[1,2,3,4,5,6,7], sheet_name=sheet_name, names=names)

            day_data['date'] = report_dt

            data = pd.concat([data, day_data])
        except IndexError:
            print('IndexError')

    data = data.reset_index(drop=True)

    return data

In [132]:
def load_files(files, func):
    data = func(files[0])
    for item in range(1, len(files)):
        df = func(files[item])
        data = pd.concat([data, df])
    
    data = data.reset_index(drop=True)

    return data
        

In [127]:
ssps_files  = [f for f in listdir(path) if filter(f, key_word='ССПС')]
apvo_files  = [f for f in listdir(path) if filter(f, key_word='АПВО')]

In [51]:
apvo_data = load_files(apvo_files, read_apvo)

File name: ../data/raw/АПВО октябрь 19.xls, first day: 2019-10-01 00:00:00, max days in month: 31
File name: ../data/raw/АПВО июль 20.xls, first day: 2020-07-01 00:00:00, max days in month: 31
File name: ../data/raw/АПВО август 20.xlsx, first day: 2020-08-01 00:00:00, max days in month: 31
File name: ../data/raw/АПВО апрель 19.xls, first day: 2019-04-01 00:00:00, max days in month: 30
File name: ../data/raw/АПВО август 20.xls, first day: 2020-08-01 00:00:00, max days in month: 31
File name: ../data/raw/АПВО июнь 20.xls, first day: 2020-06-01 00:00:00, max days in month: 30
File name: ../data/raw/АПВО июнь 19.xls, first day: 2019-06-01 00:00:00, max days in month: 30
File name: ../data/raw/АПВО апрель 20.xls, first day: 2020-04-01 00:00:00, max days in month: 30
File name: ../data/raw/АПВО сентябрь 20.xlsx, first day: 2020-09-01 00:00:00, max days in month: 30
File name: ../data/raw/АПВО май 19.xls, first day: 2019-05-01 00:00:00, max days in month: 31
File name: ../data/raw/АПВО май 

In [52]:
apvo_data

,company,type,volume,work_area,work_kind,work_done_phis,work_done_pas,window,date
0,"ПЧ-23, ШЧ-9, ДС ЯКУШКА, ДСП АМИРОВКА",Унимат 08-275/3S№ 00000040,,АМИРОВКА СТР 1,Работа УНИМАТ,1.0 стрелка,,120/0,2019-10-01
1,"ПЧ-23, ШЧ-9, ДС ПОГРУЗНАЯ, ДСП Кармала",Унимат 08-275/3S№ 00000040,,"КАРМАЛА , п. 1 СТР 2",Работа УНИМАТ,1.0 стрелка,,90/0,2019-10-01
2,"ПЧ-23, ШЧ-9, ДС ЯКУШКА, ДСП Б/П 1028 КМ",Унимат 08-275/3S№ 00000040,,ОП 1028 КМ СТР 1,Работа УНИМАТ,1.0 стрелка,,120/0,2019-10-01
3,"ПЧ-14, РЦС-3, ШЧ-6, ЭЧ-7",ДУОМАТИК 09-32 GSM№ 00000035,,"АСЕКЕЕВО-ФИЛИППОВКА , п.2 1310пк8-1329пк10",Работа ДУОМАТИК,1000.0 м,,120/0,2019-10-01
4,"ПЧ-14, РЦС-3, ШЧ-6, ЭЧ-7",РПБ-01№ 00000108,,"АСЕКЕЕВО-ФИЛИППОВКА , п.2 1310пк8-1329пк10",Работа РПБ,1000.0 м,,120/0,2019-10-01
...,...,...,...,...,...,...,...,...,...
41531,"ПЧ-20, ДС САРАНСК, ДС ЕЛОХОВКА",ПМА-1М№ 00000012,,,,,,НЕТ ЗАЯВОК,2020-05-31
41532,"ПЧ-20, ДС САРАНСК, ДС ЕЛОХОВКА",РПБ-01№ 00000051,,,,,,НЕТ ЗАЯВОК,2020-05-31
41533,"ПЧ-20, ДС САРАНСК, ДС ЕЛОХОВКА",ПМА-1М№ 00000012,,,,,,НЕТ ЗАЯВОК,2020-05-31
41534,"ПЧ-20, ДС САРАНСК, ДС ЕЛОХОВКА",ПМА-1М№ 00000012,,,,,,НЕТ ЗАЯВОК,2020-05-31


In [161]:
ssps_data = load_files(ssps_files, read_ssps)

File name: ../data/raw/работа ССПС август 20.xlsx, first day: 2020-08-01 00:00:00, max days in month: 31
File name: ../data/raw/работа ССПС сентябрь 20.xlsx, first day: 2020-09-01 00:00:00, max days in month: 30
File name: ../data/raw/работа ССПС май 19.xlsx, first day: 2019-05-01 00:00:00, max days in month: 31
File name: ../data/raw/работа ССПС июнь 20.xls, first day: 2020-06-01 00:00:00, max days in month: 30
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
File name: ../data/raw/работа ССПС июль 20.xlsx, first day: 2020-07-01 00:00:00, max days in month: 31
File name: ../data/raw/работа ССПС апрель 20.xlsx, first day: 2020-04-01 00:00:00, max days in month: 30
File name: ../data/raw/работа ССПС апрель 19.xlsx, first day: 2019-04-01 00:00:00, max days in month: 30
File name: ../data/raw/работа ССПС июнь 19.xlsx, f

In [162]:
ssps_data

,company,machine,au12,rate_norm,rate_fact,low_rate,up_rate,residue,date
0,1,ЩОМ-1200 № 16,1,-,-,-,-,622,2020-08-01
1,2,ЩОМ-1200М № 19,1,-,-,-,-,1395,2020-08-01
2,3,ТЭС-1000 № 3,1,-,-,-,-,2243,2020-08-01
3,4,ТЭС-1000 № 16,1,-,-,-,-,2268,2020-08-01
4,5,ТЭС-1000 № 19,1,-,-,-,-,3004,2020-08-01
...,...,...,...,...,...,...,...,...,...
31600,46,СМ-2 № 1596,1,-,-,-,-,5,2019-09-30
31601,47,СМ-2 № 1643,1,-,-,-,-,76,2019-09-30
31602,48,СМ-2 № 1764,1,-,-,-,-,102,2019-09-30
31603,49,ТЭС ПСС-1К № 62,1,267.4,254,13.4,-,566,2019-09-30


In [178]:
ssps_data.to_csv('../data/prep/raw/ssps_data.csv', index=False)
apvo_data.to_csv('../data/prep/raw/apvo_data.csv', index=False)

In [174]:
ssps_data.to_excel('../data/prep/raw/ssps_data.xlsx', index=False)

In [176]:
apvo_data.to_excel('../data/prep/raw/apvo_data.xlsx', index=False, engine='xlsxwriter')